This notebook is intented to create and explore different ratio calulations. Resulting processes/functions will be copied to ./dat/

In [8]:
# Set matplotlib for plotting in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

from fooof import FOOOF
from fooof import FOOOFGroup
import numpy as np
from utils.ratios import *
from fooof.utils import trim_spectrum
# Import some utilities for synthesizing some test data
from fooof.synth import gen_group_power_spectra, param_sampler, gen_power_spectrum

## Simulate Power Spectra

In [9]:
#Creates a single adhd simulated spectrum
# Alpha peaks on lower park of band to potentially bleed over to theta band

def gen_treatment_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 10)) #slow Alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [10]:
#Creats a single control simulated spectrum (Theta, slow peak alpha, beta)
def gen_control_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 12)) #slow alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [11]:
#Gen group of ADHD PSDs (slow alpha peak freq) + control PSD
bg = [0,1]

#100 trails for each adhd and control sim
treatmentFreq, treatmentPower, _ = gen_group_power_spectra(100, [1,50], bg,gen_treatment_sample(), nlvs=np.random.uniform(.005,.02)) 
contFreq, contPower, _ = gen_group_power_spectra(100, [1,50], bg, gen_control_sample(), nlvs=np.random.uniform(.005,.02))

treatment = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)
control = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)

In [12]:
treatment.fit(treatmentFreq,treatmentPower)
control.fit(contFreq,contPower)

In [13]:
#Gather peak info for both PSDs
treatment_peaks = treatment.get_all_data('peak_params')
control_peaks = control.get_all_data('peak_params')

## Calculate theta/beta ratios 
#### difference in power of central frequency of high and low frequency peak only

In [7]:
# treatment (Slow alpha peak)
treatmentRatioList = []
for i in range(len(treatment_peaks)-2):
    treatmentRatioList.append(treatment_peaks[i][1]/treatment_peaks[i+2][1])
    i+=2

In [15]:
#Sanity Check
treatmentRatioList[0]

0.9695096373745006

In [16]:
# control
controlRatioList = []
for i in range(len(control_peaks)-2):
    controlRatioList.append(control_peaks[i][1]/control_peaks[i+2][1])
    i+=2

In [17]:
#Sanity Check
controlRatioList[0]

0.6144016229112761

In [12]:
#Difference in mean ratios of treatment sim and control sim
abs(np.mean(treatmentRatioList)-np.mean(controlRatioList))

0.00568426914238751

### Difference in - average low band power / average high band power (bands set)

In [20]:
treatment_ratio = calc_theta_beta_ratio(treatmentFreq,treatmentPower)
control_ratio = calc_theta_beta_ratio(contFreq,contPower)
print("treatment ratio: ",treatment_ratio)
print("control ratio: ", control_ratio)

treatment ratio:  6.58077589748655
control ratio:  4.4580031201437045


In [19]:
treatment_ratio - control_ratio

2.1227727773428455

### Difference in - sum(powers in band)/bandwidth ratio

In [66]:
def calc_band_ratio_sum(freqs, psd, low_band_range, high_band_range):
    
    # Extract frequencies within each specified band
    _, low_band_powers = trim_spectrum(freqs, psd, low_band_range)
    _, high_band_powers = trim_spectrum(freqs, psd, high_band_range)
   
    # Divides each sum of each power spectrum by bandwidth.
    low_sum = sum(low_band_powers) / len(low_band_range) 
    high_sum = sum(high_band_powers) / len(low_band_range)
    
    # Average divided sums and find ratio
    return np.mean(low_sum)/np.mean(high_sum)
    

In [67]:
calc_band_ratio_sum(treatmentFreq,treatmentPower,[4,8],[15,30])


6.580775897486551